In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
dirNameInput = os.getcwd() + '/Word2VecData_Input'
dirNameOutput = os.getcwd() + '/Word2VecData_Output'

from sklearn.model_selection import train_test_split

Word2Vec_model = None

X = None
Y = None
X_Actual = None
Y_Actual = None
X_Simulated = None
Y_Simulated = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None
result = []


In [2]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    
    fileNamesActual = ['autopark.dat', 'drive.dat', 'idle.dat']
    
    # To only work with simulated data start commenting from here
    fileInput = open("Word2VecData_Input\Word2VecDataActual.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataActual.txt", "w")
    

    for i in range(len(fileNamesActual)):
        with open(fileNamesActual[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]
        value = int(0.8 * len(content))
        
        value = len(content)

        cntCntr = 0
        while cntCntr < value:
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < value:
                parts = content[cntCntr].split(",")

                # this is the whole message
                DATA = parts[3].split(":")[1].replace(" ", "")

                currMessagePackets += str(DATA) + " "

                cntCntr += 1
                iteration += 1

            if cntCntr < value:
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(i) + "\n")
    fileInput.close()
    fileOutput.close()
    # To only work with Simulated data end commenting till here

    # To only work with actual data start commenting from here
    # For Simulated data
    fileInput = open("Word2VecData_Input\Word2VecDataSimulated.txt", "w")
    fileOutput = open("Word2VecData_Output\Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['New_DRIVE_Data.txt', 'New_IDLE_Data.txt', 'DRIVE.rtf', 'IDLE.rtf']
    arrSimulated = [1, 2, 1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < 5 and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)


In [3]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [4]:
def train():
    createDataFiles()
    createWord2VecModels()

In [5]:
train()

Awesome !!! File processing done !!!
Total Time for file processing --->  1.0129814147949219


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  1.7073695659637451


In [6]:

# Convert the data into Numpy Arrays
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
x_train = None
def getNumpyArray():
    global X
    global Y
    global X_Actual
    global Y_Actual
    global X_Simulated
    global Y_Simulated
    global dirNameInput
    X = None
    Y = None
    result = []
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            currV = Word2Vec_model.wv[parts[0]]
            currV.setflags(write=1)
            for i in range(1, len(parts)):
                currV += Word2Vec_model.wv[parts[i]]
            currV /= len(parts)
            result.append(np.array(currV))
        if "Actual" in fName: 
            X_Actual = np.array(result)
            result = []
        else:
            X_Simulated = np.array(result)
            result = []
    
    X = np.concatenate((X_Actual, X_Simulated), axis=0)
                        
    result = []
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            result.append([currV])
        if "Actual" in fName:
            Y_Actual = np.array(result)
            result = []
        else:
            Y_Simulated = np.array(result)
            result = []
                
    Y = np.concatenate((Y_Actual, Y_Simulated), axis=0)
    
    print(X.shape)
    print(Y.shape)
    end = time.time()
    print("Total time for array conversion -> ", end -start)

In [7]:
def createDNNModel():
    global model     
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy']) 

In [8]:

getNumpyArray()
# Comment this line for your part
createDNNModel()



(26777, 200)
(26777, 1)
Total time for array conversion ->  1.0790808200836182


In [9]:
def executeModel(eph, X_arr, Y_arr):
    start = time.time()
    x_train, x_test, y_train, y_test = train_test_split(X_arr, Y_arr,test_size=0.30, random_state=1)
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=eph)
    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)

In [13]:
# Simulated + Actual
start = time.time()
executeModel(30, X, Y)
end = time.time()
print("Total Time -->", end - start)

Epoch 1/30
18743/18743 [==============================] - 1s 78us/step - loss: 0.1228 - acc: 0.9754

In [16]:
# Simulated
start = time.time()
executeModel(30, X_Simulated, Y_Simulated)
end = time.time()
print("Total Time for all data -->", end - start)

Epoch 1/30
14832/14832 [==============================] - 1s 92us/step - loss: 0.2519 - acc: 0.9048

In [17]:
start = time.time()
executeModel(30, X_Actual, Y_Actual)
end = time.time()
print("Total Time for Actual data-->", end - start)

Epoch 1/30
3911/3911 [==============================] - 1s 203us/step - loss: 0.4539 - acc: 0.9322
Epoch 2/30
3911/3911 [==============================] - 0s 88us/step - loss: 0.0437 - acc: 0.9877

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()